# App Profile Recommendation

In this project we analyze the data available from our various apps to find out user engagement. As all of our apps are free, the main revenue stream is trhough in-app ads. The more the user interacts with theem, the more profitable the apps become.

The main goal of the project is to find out those apps that are generating most revenue and/or show the most potential for generating higher revenue.